In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 Vertex AI Vizier 开始

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vizier/get_started_vertex_vizier.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      查看 GitHub 上的内容
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vizier/get_started_vertex_vizier.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/vizier/get_started_vertex_vizier.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>           
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex AI Vizier进行超参数调整。

了解更多关于[Vertex AI Vizier](https://cloud.google.com/vertex-ai/docs/vizier/overview)的信息。

### 目标

在本教程中，您将学习如何在使用`Vertex AI`时使用`Vertex AI Vizier`。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI 训练`
- `Vertex AI 超参数调整`
- `Vertex AI Vizier`

执行的步骤包括：

- 使用随机算法进行超参数调整。
- 使用 Vizier（贝叶斯）算法进行超参数调整。
- 为 Vizier 研究建议试验并更新结果。

### 推荐

在谷歌云上进行端到端的MLOps时，以下是在什么时候使用Vertex AI Vizier进行超参数调整的最佳实践：

**网格搜索**

当您有少量离散值的组合时使用。例如，您有以下两个超参数和离散值：

- 批大小: \[ 16, 32, 64\]
- 学习率: \[ 0.001, 0.01. 0.1\]

组合总数为9（3 x 3）。

**随机搜索**

当您有少量超参数，其中至少一个是连续值时使用。例如，您有以下两个超参数和范围：

- 批大小: \[ 16, 32, 64\]
- 学习率: 0.001 .. 0.1

**Vizier搜索**

当您有：

- 大量的超参数和离散值
- 巨大的连续搜索空间
- 多个目标时使用

### 数据集

本教程使用的数据集是[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。本教程中使用的数据集版本已经内置在TensorFlow中。训练好的模型可以预测房屋的中位价格，单位为1K美元。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 的定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 的定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 基于您的预期使用量生成成本估计。

## 安装

安装所需的包以执行这个笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-vizier==0.0.4  

### 只有共同协作：取消下面的单元格注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在你开始之前

### 设置您的项目 ID

**如果您不知道您的项目 ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud帐户

根据您的Jupyter环境，您可能需要手动进行验证。请按照以下相关说明进行操作。

1. Vertex AI 工作台
* 无需操作，因为您已经通过身份验证。

2. 本地JupyterLab实例，取消注释并运行：

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账号或其他
* 查看如何将云存储权限授予您的服务账号，请访问https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

### 创建一个云存储桶

创建一个存储桶，用于存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-unique-{PROJECT_ID}"  # @param {type:"string"}

只有当您的存储桶尚不存在时，才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

创建变量

接下来，设置一些在教程中使用的变量。
导入库并定义常量。

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud.aiplatform.vizier import Study, pyvizier

初始化用于Python的Vertex AI SDK

为您的项目和对应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练设置硬件加速器。

设置变量`TRAIN_GPU / TRAIN_NGPU`以使用支持GPU的容器映像并分配给虚拟机（VM）实例的GPU数量。例如，要使用一个带有4个Nvidia Telsa K80 GPU的GPU容器映像，您可以指定：

（aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80，4）

否则，指定`(无，无)`以使用一个在CPU上运行的容器映像。

在您的地区了解更多关于硬件加速器支持的信息[这里]（https://cloud.google.com/vertex-ai/docs/general/locations#accelerators）。

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### 设置预先构建的容器

设置用于训练的预先构建的Docker容器镜像。

- 将变量`TF`设置为容器镜像的TensorFlow版本。例如，`2-1`表示版本2.1，`1-15`表示版本1.15。以下列表显示一些可用的预构建镜像：

要获取最新列表，请参阅[用于训练的预建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

In [ ]:
TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

#### 设置机器类型

接下来，设置用于训练的机器类型。

- 将变量`TRAIN_COMPUTE`设置为配置用于训练的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存。
     - `n1-highcpu`：每个vCPU 0.9GB内存。
 - `vCPUs`：数量为 \[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下内容不支持用于训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
TRAIN_COMPUTE = "n1-standard-4"
print("Train machine type", TRAIN_COMPUTE)

## 独立的 Vertex AI Vizer 服务

`Vizier` 服务可以作为一个独立服务用于选择试验的下一个参数集。

*注意：* 该服务不执行试验。您需要创建自己的试验和执行。

了解更多关于[Vizier 的使用](https://cloud.google.com/vertex-ai/docs/vizier/using-vizier)。

## Vertex AI 超参数调整服务

以下示例演示了如何使用 Vertex AI 超参数调整服务和 `random` 搜索算法设置、执行和评估试验。

了解更多关于[超参数调整概述](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)。

### 检查超参数调整包

#### 包布局

在开始超参数调整之前，您将查看如何为自定义超参数调整作业组装一个Python包。解压后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境中的说明书。

文件`trainer/task.py`是用于执行自定义超参数调整作业的Python脚本。*注意*，当我们在工作进程池规范中提到它时，我们会用点 (`trainer.task`) 替换目录斜杠，并省略文件后缀 (`.py`)。

#### 包装配

在下面的单元格中，您将组装训练包。

In [ ]:
# Make folder for Python hyperparameter tuning script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow==2.5.0',\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration hyperparameter tuning script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 文件内容

在下一个单元格中，您将编写超参数调整脚本 task.py 的内容。我不会详细介绍，只是让您浏览一下。总结如下：

- 解析当前试验的超参数设置的命令行参数。
- 从命令行（`--model_dir`）或环境变量`AIP_MODEL_DIR`获取保存模型构件的目录，如果没有指定的话。
- 下载并预处理波士顿房价数据集。
- 构建一个 DNN 模型。
- 使用每个稠密层的单元数量和学习率超参数值来构建和编译模型。
- 定义一个回调 `HPTCallback`，该回调在每个周期结束时获得验证损失，并使用 `hpt.report_hyperparameter_tuning_metric()` 报告给超参数调整服务。
- 使用 `fit()` 方法训练模型，并指定一个回调，将验证损失报告给超参数调整服务。

In [ ]:
%%writefile custom/trainer/task.py
# Custom Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
from hypertune import HyperTune
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--decay', dest='decay',
                    default=0.98, type=float,
                    help='Decay rate')
parser.add_argument('--units', dest='units',
                    default=64, type=int,
                    help='Number of units.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)

# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(args.units, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(args.units, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr, decay=args.decay))
  return model


model = build_and_compile_dnn_model()

# Instantiate the HyperTune reporting object
hpt = HyperTune()

# Reporting callback
class HPTCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        global hpt
        hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='val_loss',
        metric_value=logs['val_loss'],
        global_step=epoch)

# Train the model
BATCH_SIZE = 16
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=BATCH_SIZE, validation_split=0.1, callbacks=[HPTCallback()])
model.save(args.model_dir)

将超参数调整脚本存储在您的云存储桶中

接下来，将超参数调整文件夹打包成压缩的tar存档文件，然后将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_boston.tar.gz

准备您的机器规格

现在定义您定制的超参数调整作业的机器规格。这会告诉 Vertex 需要为超参数调整提供什么类型的机器实例。
- `machine_type`：要提供的 GCP 实例类型，例如 n1-standard-8。
- `accelerator_type`：硬件加速器的类型（如果有的话）。在本教程中，如果您之前设置了变量`TRAIN_GPU != None`，则您正在使用 GPU；否则将使用 CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在定义您自定义超参数调整作业的磁盘规格。这将告诉 Vertex 每台机器实例为超参数调整提供什么类型和大小的磁盘。

- `boot_disk_type`：SSD或标准。SSD更快，标准更便宜。默认为SSD。
- `boot_disk_size_gb`：磁盘大小（GB）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

###定义工作池规范

接下来，您需要为自定义超参数调整作业定义工作池规范。工作池规范将包括以下内容：

- `replica_count`：要配置的此机型的实例数量。
- `machine_spec`：硬件规范。
- `disk_spec`：（可选）磁盘存储规范。

- `python_package`：要安装在VM实例上的Python训练包，并指定要调用的Python模块，以及Python模块的命令行参数。

现在让我们深入了解Python包规范：

- `executor_image_spec`：这是为您的自定义超参数调整作业配置的docker镜像。

- `package_uris`：这是您的Python训练包要安装到已配置实例上的位置（URI）列表。这些位置需要在Cloud Storage存储桶中。这些可以是单个Python文件，也可以是整个包的zip（归档）文件。在后一种情况下，作业服务将解压缩（解档）内容到docker镜像中。

- `python_module`：要调用以运行自定义超参数调整作业的Python模块（脚本）。在此示例中，您将调用`trainer.task.py` -- 请注意不需要附加`.py`后缀。

- `args`：要传递给相应Python模块的命令行参数。在此示例中，您将设置：
   - `"--model-dir=" + MODEL_DIR`：存储模型工件的Cloud Storage位置。有两种告诉超参数调整脚本保存模型工件的位置的方法：
       - 直接：将Cloud Storage位置作为命令行参数传递给训练脚本（设置变量`DIRECT=True`），或
       - 间接：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给训练脚本（设置变量`DIRECT=False`）。在这种情况下，您需要在作业规范中告诉服务模型工件位置。
   - `"--epochs=" + EPOCHS`：用于训练的时期数。
   - `"--steps=" + STEPS`：每个时期的步数（批次）。
   - `"--distribute=" + TRAIN_STRATEGY"`：用于单个或分布式超参数调整的超参数调整分发策略。
      - `"single"`：单个设备。
      - `"mirror"`：单个计算实例上的所有GPU设备。
      - `"multi"`：所有计算实例上的所有GPU设备。

In [ ]:
JOB_NAME = "custom_job"
MODEL_DIR = "{}/{}".format(BUCKET_URI, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = False
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_URI + "/trainer_boston.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

创建自定义作业

使用`CustomJob`类来创建一个自定义作业，比如用于超参数调整，具有以下参数：

- `display_name`：自定义作业的人类可读名称。
- `worker_pool_specs`：相应 VM 实例的规格。

In [ ]:
job = aiplatform.CustomJob(display_name="boston", worker_pool_specs=worker_pool_spec)

## 创建一个超参数调优作业

使用类`HyperparameterTuningJob`来创建一个超参数调优作业，具有以下参数：

- `display_name`: 用于自定义作业的人类可读的名称。
- `custom_job`: 此自定义作业中的工作池规范适用于所有试验中创建的CustomJobs。
- `metrics_spec`: 要优化的指标。字典键是由训练作业报告的指标ID，字典值是指标的优化目标（'minimize'或'maximize'）。
- `parameter_spec`: 要优化的参数。字典键是作为命令行关键字参数传递给您的训练作业的指标ID，字典值是指标的参数规范。
- `search_algorithm`: 要使用的搜索算法：`grid`、`random`和`None`。如果指定了`None`，则使用`Vizier`服务（贝叶斯）。
- `max_trial_count`: 要执行的最大试验次数。

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

hpt_job = aiplatform.HyperparameterTuningJob(
    display_name="boston",
    custom_job=job,
    metric_spec={
        "val_loss": "minimize",
    },
    parameter_spec={
        "lr": hpt.DoubleParameterSpec(min=0.001, max=0.1, scale="log"),
        "units": hpt.IntegerParameterSpec(min=4, max=128, scale="linear"),
    },
    search_algorithm="random",
    max_trial_count=6,
    parallel_trial_count=1,
)

运行超参数调整工作

使用`run()`方法执行超参数调整工作。

In [ ]:
hpt_job.run()

显示超参数调整作业试验结果

超参数调整作业完成后，属性`trials`将返回每个试验的结果。

In [ ]:
print(hpt_job.trials)

现在看看哪项试验是最好的：

In [ ]:
best = (None, None, None, 0.0)
for trial in hpt_job.trials:
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

print(best)

## 获取最佳模型

如果您使用了让服务告诉调优脚本在何处保存模型工件的方法（`DIRECT = False`），则最佳模型的模型工件将保存在：

    MODEL_DIR/<best_trial_id>/model

In [ ]:
BEST_MODEL_DIR = MODEL_DIR + "/" + best[0] + "/model"

### 删除超参数调整作业

方法 'delete()' 将删除超参数调整作业。

In [ ]:
hpt_job.delete()

## Vertex AI超参数调整与Vertex AI Vizer服务结合

以下示例演示如何使用Vertex AI超参数调整服务和`Vizier`搜索服务设置、执行和评估试验。

创建自定义作业

使用`CustomJob`类创建自定义作业，例如用于超参数调整，具有以下参数：

- `display_name`：自定义作业的人类可读名称。
- `worker_pool_specs`：相应VM实例的规范。
- `base_output_dir`：用于存储模型工件的云存储位置。

In [ ]:
job = aiplatform.CustomJob(
    display_name="boston",
    worker_pool_specs=worker_pool_spec,
    base_output_dir=MODEL_DIR,
)

创建超参数调优作业

使用`HyperparameterTuningJob`类创建一个超参数调优作业，具有以下参数:

- `display_name`: 为自定义作业指定一个易于识别的名称。
- `custom_job`: 该自定义作业中的工作池规范适用于所有试验中创建的CustomJobs。
- `metrics_spec`: 要优化的指标。字典键是由您的训练作业报告的metric_id，字典值是指标的优化目标（'minimize'或'maximize'）。
- `parameter_spec`: 要优化的参数。字典键是传递到您的训练作业中作为命令行关键字参数的metric_id，字典值是该指标的参数规范。
- `search_algorithm`: 要使用的搜索算法: `grid`、`random`和`None`。如果指定`None`，则使用`Vizier`服务（贝叶斯）。
- `max_trial_count`: 要执行的最大试验次数。

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

hpt_job = aiplatform.HyperparameterTuningJob(
    display_name="boston",
    custom_job=job,
    metric_spec={
        "val_loss": "minimize",
    },
    parameter_spec={
        "lr": hpt.DoubleParameterSpec(min=0.0001, max=0.1, scale="log"),
        "units": hpt.IntegerParameterSpec(min=4, max=512, scale="linear"),
    },
    search_algorithm=None,
    max_trial_count=12,
    parallel_trial_count=1,
)

运行超参数调整作业

使用`run()`方法来执行超参数调整作业。

In [ ]:
hpt_job.run()

### 显示超参数调整作业试验结果

超参数调整作业完成后，属性 `trials` 将返回每次试验的结果。

In [ ]:
print(hpt_job.trials)

现在看看哪个试验是最好的：

In [ ]:
best = (None, None, None, 0.0)
for trial in hpt_job.trials:
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

print(best)

获取最佳模型

如果您使用的方法是让服务告诉调整脚本在哪里保存模型工件（`DIRECT = False`），那么最佳模型的模型工件将保存在：

MODEL_DIR/<best_trial_id>/model

In [ ]:
BEST_MODEL_DIR = MODEL_DIR + "/" + best[0] + "/model"

! gsutil ls {BEST_MODEL_DIR}

删除超参数调整作业

方法“delete()”将删除超参数调整作业。

In [ ]:
hpt_job.delete()

## 单独使用的 Vertex AI Vizer 服务

`Vizier` 服务可以作为一个独立的服务，用于选择下一个试验的参数设置。

*注意:* 该服务不执行试验。您需要创建自己的试验和执行。

了解更多关于 [使用 Vizier](https://cloud.google.com/vertex-ai/docs/vizier/using-vizier)

### 指定用于建议试验参数的算法

首先，您需要创建一个`StudyConfig`，并指定算法来建议下一个试验。

    GRID_SEARCH: 网格搜索
    RANDOM_SEARCH: 随机搜索
    ALGORITHM_UNSPECIFIED: Vizier 贝叶斯算法

In [ ]:
problem = pyvizier.StudyConfig()
problem.algorithm = pyvizier.Algorithm.RANDOM_SEARCH

创建一个研究

研究是一系列的实验或试验，可以帮助您优化超参数或参数。

在以下示例中，目标是最大化 y = x^2，其中 x 在范围 \[-10, 10\] 内。这个示例只有一个参数，并使用一个易于计算的函数来帮助演示如何使用Vizier。

首先，您将要最小化或最大化的指标以列表形式指定为 `metric_information` 属性。然后，使用 `add_XXX_params()` 方法为相应数据类型指定研究参数：

    - add_bool_param
    - add_categorical_param
    - add_discrete_param
    - add_float_param
    - add_int_param

您可以使用 `create_or_load()` 方法来创建研究。

In [ ]:
STUDY_DISPLAY_NAME = "xpow2"

problem.metric_information.append(
    pyvizier.MetricInformation(name="y", goal=pyvizier.ObjectiveMetricGoal.MAXIMIZE)
)

params = problem.search_space.select_root()
params.add_float_param("x", -10.0, 10.0, scale_type=pyvizier.ScaleType.LINEAR)

study = Study.create_or_load(display_name=STUDY_DISPLAY_NAME, problem=problem)

STUDY_NAME = study.name
print("STUDY_NAME: {}".format(STUDY_NAME))

获取Vizier研究

您可以使用`list()`方法获取研究。

In [ ]:
studies = Study.list()
print(studies[0].gca_resource)

获取建议试验

接下来，使用方法`suggest()`查询Vizier服务以获取建议试验，包括以下键/值对：

- `count`：建议的试验数量。

这个调用是一个长时间运行的操作。响应对象的`result()`方法会等待调用完成。

In [ ]:
SUGGEST_COUNT = 1

trials = study.suggest(count=SUGGEST_COUNT)

print(trials)

# Get the trial ID of the first trial
TRIAL_ID = trials[0].name

### 评估结果
在收到您的试验建议后，评估每个试验并将每个结果记录为一项测量。

例如，如果您要优化的函数是 y = x^2，则使用试验中建议的 x 值评估该函数。使用建议值为 0.1，函数评估结果为 y = 0.1 * 0.1，得到 0.01。

### 添加一个测量

在评估您的试验建议以获取一个测量之后，请将此测量添加到您的试验中。

请使用以下命令来存储您的测量并发送请求。在这个例子中，用测量值替换 RESULT。如果您正在优化的函数是 y = x^2，并且建议的 x 值为 0.1，那么结果就是 0.01。

In [ ]:
RESULT = 0.01

measurement = pyvizier.Measurement()
measurement.metrics["y"] = RESULT

trials[0].add_measurement(measurement)

### 删除大臣研究

方法 'delete()' 将删除这项研究。

In [ ]:
study.delete()

清理

要清理此项目中使用的所有Google Cloud资源，可以[删除您用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

- 云存储存储桶

In [ ]:
import os

delete_bucket = False

if os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI